In [1]:
%load_ext autoreload
%autoreload 2

In [38]:
import json
import os
import time
import glob
import numpy as np

In [182]:
os.cpu_count()

24

In [42]:
from screwing_dataset import ScrewingDataset
from screwing_model import ScrewingModel
from screwing_model_seq import ScrewingModelSeq


In [43]:
from torch.utils.data import DataLoader, ConcatDataset
import torch
import torch.nn as nn
import torch.optim as optim

In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [56]:
peg_r = 0.0127
half_peg_r = peg_r/2


range(5)

range(0, 5)

In [36]:
list_test = [1.1, 2, 3]
tensor_test = torch.as_tensor(list_test)
q = torch.as_tensor([0.1, 0.5, 0.9])
intervals = torch.quantile(tensor_test, q, dim=0, keepdim=False, interpolation='nearest')
print(intervals[1].item())
print(interval_lower)

Failed to load Python extension for LZ4 support. LZ4 compression will not be available.
[autoreload of rospy.impl.init failed: Traceback (most recent call last):
  File "/home/serialexperimentsleon/.pyenv/versions/3.7.11/envs/pytorch/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/serialexperimentsleon/.pyenv/versions/3.7.11/envs/pytorch/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/serialexperimentsleon/.pyenv/versions/3.7.11/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/serialexperimentsleon/.pyenv/versions/3.7.11/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._b

2.0
tensor([1.1000, 2.0000, 3.0000])


In [52]:
base_dset_dir = os.path.expanduser('~/datasets/screwing')

# xprmnt_dir = time.strftime("/%Y-%m-%d_%H-%M-%S")
xprmnt_dir = time.strftime("/2022-03-10_23-17-39")


In [53]:
bag_path_names = base_dset_dir + xprmnt_dir + '/*.bag' 

bag_path_list = glob.glob(bag_path_names)
total_num_dset = len(bag_path_list)
num_dset = 2

In [54]:
dset_list = []
window_size = 10
for i in range(num_dset):
    id_str = str(i)
    bag_path_names = base_dset_dir + xprmnt_dir + '/' + id_str + '_*.bag' 
    bag_path = glob.glob(bag_path_names)[0]

    pos_path_name = base_dset_dir + xprmnt_dir + '/' + id_str + '_pos.npy'
    proj_ori_path = base_dset_dir + xprmnt_dir + '/' + id_str + '_proj_ori.npy'
    pos_ori_path_list = [pos_path_name, proj_ori_path]
    
    dset_list.append(ScrewingDataset(bag_path, pos_ori_path_list, window_size))
    
concat_dset = ConcatDataset(dset_list)

ROSBagUnindexedException: Unindexed bag

In [49]:
length = len(concat_dset)
print(length)
train_size = int(.7*length)
# train_size
train_dset, test_dset = torch.utils.data.random_split(concat_dset, [train_size,length - train_size])

NameError: name 'concat_dset' is not defined

In [9]:
batch_size = 2

train_lder = DataLoader(
    train_dset,
    shuffle=True,
    num_workers=2,
    batch_size=batch_size
)

test_lder = DataLoader(
    test_dset,
    shuffle=True,
    num_workers=2,
    batch_size=batch_size
)

In [10]:
input_dim = 19
hidden_dim = input_dim*2
num_layers = int(window_size*0.5)
output_dim = 5
model = ScrewingModel(input_dim, hidden_dim, num_layers, output_dim)

In [135]:
print(out.size())
# print(out[:, -1, :].size())

torch.Size([2, 5])


In [138]:
def training_loop(net,optimizer,criterion,num_epochs,train_loader):
  overall_step = 0
  counter = 0
  loss_plot = []
  for epoch in range(num_epochs):
    for i,(x,y) in enumerate(train_loader):
      counter += 1
      optimizer.zero_grad()

      x = x.to(device)
      y = y.float().to(device)

      # Forward propogation happens here
      outputs = net(x).to(device) 

      # The following lines of code is where the magic happens!
      # The partial derivatives of the Loss with respect to the 
      # weights are calculated and the weights are updated
      loss = criterion(outputs,y)  
      loss.backward()
      optimizer.step()
      loss_plot.append(loss.item())

      # if (counter+1) % 10 == 0: 
          # argmax = torch.max(outputs, 1)[1]
          # accuracy = (y == argmax.squeeze()).float().mean()

#           info = {'loss' : loss.item()}
#           for tag, value in info.items():
#               logger.scalar_summary(tag, value, overall_step)
#           overall_step+=1

#           print ('Epoch : %d/%d, Iter : %d/%d,  Loss: %.4f' 
#                  %(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))
  return net #, loss_plot

In [86]:
num_epochs = 10
learning_rate = 0.003

# net = My_Neural_Network()
# net = net.to(device)

# model = ScrewingModel(input_dim, hidden_dim, ouput_dim) # no specification of 
model = model.to(device)
loss_function = nn.MSELoss(reduction='sum') #alternatively mean the squared error or none ...
optimizer = optim.Adam(model.parameters(), lr=learning_rate)




In [128]:
a = torch.rand(2,5)
print(a)
print(torch.norm(a, 2, 1))
ones = torch.ones(2, 1)
print(ones)
torch.cat((a, ones), 1)

b  = torch.rand(2,5)
print(b)
a / b
# a.size()[0]
# len(a)

tensor([[0.1445, 0.2926, 0.6247, 0.4714, 0.8621],
        [0.2108, 0.5757, 0.1864, 0.8268, 0.0952]])
tensor([1.2092, 1.0504])
tensor([[1.],
        [1.]])
tensor([[0.1544, 0.5451, 0.5257, 0.3115, 0.9082],
        [0.5123, 0.3980, 0.7262, 0.3125, 0.9060]])


tensor([[0.9360, 0.5368, 1.1885, 1.5134, 0.9492],
        [0.4116, 1.4465, 0.2567, 2.6455, 0.1051]])

In [98]:
def weighted_MSE_loss(input, target, ori_weight):
    SE_out = (input - target)**2
    # print(torch.mean(SE_out))
    # print(torch.sum(SE_out)/torch.numel(input))
    pos_SE = SE_out[:, :3]
    ori_SE = SE_out[:, 3:]
    # print((torch.sum(pos_SE) + torch.sum(ori_SE))/torch.numel(input))
    weighted_mean = (torch.sum(pos_SE) + ori_weight*torch.sum(ori_SE))/len(SE_out)
    # weighted_sum = torch.sum(pos_SE) + ori_weight*torch.sum(ori_SE)
    return weighted_mean
    # return torch.sum(torch.mul(weighting, MSE_out))
    # return torch.sum(MSE_out)

In [151]:
with torch.no_grad():
    for i, (x, y) in enumerate(train_lder):
        # print(x.size())
        x = x.to(device)
        out = model(x)
        print(out)
        y = y.float().to(device)
        print(y)
        # print(torch.bmm(out, y)) 
        bvv = torch.bmm(out.view(2, 1, 5), y.view(2, 5, 1)).reshape(-1)
        print(bvv)
        print(bvv.size())
        # print((out**2).sum(1, keepdim=True))
        SE_out = torch.sqrt((out**2).sum(1, keepdim=False))
        print(SE_out)
        print(SE_out.size())
        # print(out.size()[1])
        
        print(torch.cat((bvv, SE_out)))
        
        list_batch = [bvv, SE_out]
        list_batch = torch.cat(list_batch)
        print(list_batch)
        
        ori_weighting = 5 # relative to position weighting
        MSE = nn.MSELoss(reduction='none')
        # print(weighted_MSE_loss(out, y, ori_weighting))
        break
    

tensor([[ 0.0254, -0.1393,  0.1538, -0.1316,  0.0819],
        [ 0.0253, -0.1405,  0.1519, -0.1305,  0.0827]], device='cuda:0')
tensor([[ 0.5526, -0.0125,  0.5124, -0.0292, -0.0315],
        [ 0.5529, -0.0057,  0.5125, -0.0111, -0.0009]], device='cuda:0')
tensor([0.0959, 0.0940], device='cuda:0')
torch.Size([2])
tensor([0.2603, 0.2594], device='cuda:0')
torch.Size([2])
tensor([0.0959, 0.0940, 0.2603, 0.2594], device='cuda:0')
tensor([0.0959, 0.0940, 0.2603, 0.2594], device='cuda:0')


In [ ]:
trained_model = training_loop(model,optimizer,loss_function,num_epochs,train_lder)


In [155]:
with torch.no_grad():
    for i, (x, y) in enumerate(train_lder):
        # print(x.size())
        x = x.to(device)
        out = trained_model(x)
        y = y.to(device)
        # out = out.to(device)
        print(loss_function(out, y))
        print(out)
        print(y)
        break

tensor(0.0587, device='cuda:0', dtype=torch.float64)
tensor([[ 0.5505, -0.0080,  0.5124, -0.0217, -0.0615],
        [ 0.5471,  0.0051,  0.5139, -0.0379,  0.0237]], device='cuda:0')
tensor([[ 0.5764, -0.0043,  0.5101,  0.2047,  0.0008],
        [ 0.5393,  0.0024,  0.5119, -0.0801,  0.0556]], device='cuda:0',
       dtype=torch.float64)


In [158]:
out

tensor([[ 0.5591, -0.0023,  0.5127,  0.0754, -0.0207],
        [ 0.5488,  0.0055,  0.5150, -0.0272,  0.0273]], device='cuda:0')

In [179]:
# model_save_dir = os.getcwd()
model_save_dir = '../../models'
model_name = time.strftime("/model_%Y-%m-%d_%H-%M-%S.pt")
# model_name = '/test_model.pt'
torch.save(trained_model.state_dict(), model_save_dir + model_name)

In [178]:
torch.save(trained_model.state_dict(), model_save_dir + model_name)


In [173]:
load_model = ScrewingModel(input_dim, hidden_dim, num_layers, output_dim)
load_model.load_state_dict(torch.load(model_save_dir + model_name))
load_model.eval()

ScrewingModel(
  (lstm): LSTM(19, 38, num_layers=5, batch_first=True)
  (hidden2out): Linear(in_features=38, out_features=5, bias=True)
)